# Decision Trees, Part 2 - Feature Selection and Missing Data

In this notebook, we'll dive deeper into two important aspects of Decision Trees: Feature Selection and handling Missing Data. 

## 1. Feature Selection
Feature selection is the process by which we automatically or manually select those features which contribute most to our prediction variable or output in which we are interested in. Having irrelevant features in our data can decrease the accuracy of many models, especially linear algorithms like linear and logistic regression.

### 1.1 Automatic Feature Selection
In decision trees, feature selection happens automatically. This is because at each node of the tree, the model evaluates every feature to determine which one will result in the greatest reduction in impurity, and this feature is the one used to make the split at that node. If a feature never results in a meaningful reduction in impurity, it will not be used to make any decisions in the tree.

Let's illustrate this with an example. Suppose we have a dataset about patients' health, including symptoms like good blood circulation, blocked arteries, and chest pain, and a target variable indicating whether each patient has heart disease. We build a decision tree and find that the chest pain feature never gives us a significant reduction in impurity. In that case, chest pain will not be part of our tree, even though we have data for it. This is a form of automatic feature selection.

![Feature Selection](https://miro.medium.com/max/1200/0*Q8zGa2eQjfJ5f3P-.png)

### 1.2 Thresholding for Feature Selection
We can also introduce a threshold such that the impurity reduction has to be large enough to make a significant difference. This results in simpler trees and prevents overfitting. Overfitting is when our model does well with the original data (the data we used to make the tree) but doesn't perform well with any other dataset. Decision trees often suffer from overfitting, and setting a threshold for impurity reduction can help mitigate this issue.

## 2. Handling Missing Data
Sometimes, we don't have data for every feature for every observation. But missing data doesn't need to be a problem for decision trees. There are several strategies we can use to deal with missing data.

### 2.1 Common Option for Categorical Data
If we're dealing with a categorical feature, we could pick the most common category. For example, if the 'blocked arteries' feature is missing for a patient, and the majority of patients in the dataset have 'Yes' for this feature, we could fill in the missing value with 'Yes'.

### 2.2 Using Correlated Features
Alternatively, we could find another feature that has a high correlation with the feature we're missing and use it as a guide. For example, if we find that 'chest pain' and 'blocked arteries' are often similar (if a patient has chest pain, they often also have blocked arteries, and vice versa), we could use the value of the 'chest pain' feature to fill in the missing 'blocked arteries' value.

### 2.3 Mean/Median or Regression for Continuous Data
For continuous features, we could replace a missing value with the mean or median of the known values for that feature. Alternatively, if another feature is highly correlated with the feature we're missing, we can do a linear regression with the two features and use the least squares line to predict the missing value. For example, if we're missing a patient's weight, but we have their height (and height is highly correlated with weight), we could use the height to predict the missing weight.

In the following sections, we will illustrate these concepts with some examples using Python and the scikit-learn library. Let's get started!

In [ ]:
# Start by importing the necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree

## 1. Feature Selection
Let's start by demonstrating feature selection. We will use the 'Pima Indians Diabetes Database' dataset from the UCI Machine Learning Repository. This dataset has 8 features and a binary target variable indicating whether a patient has diabetes. Some of the features are more predictive of diabetes than others, so we should see the decision tree primarily using those features and ignoring the less predictive ones.


In [ ]:
# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
data = pd.read_csv(url, names=names)

# Split the data into features and target variable
X = data.values[:, 0:8]
Y = data.values[:,8]

# Split the dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 100)

# Create Decision Tree classifier object and fit it to the training data
clf_entropy = DecisionTreeClassifier(criterion = "entropy", random_state = 100,
 max_depth=3, min_samples_leaf=5)
clf_entropy.fit(X_train, y_train)

# Make predictions on the test data
y_pred = clf_entropy.predict(X_test)
y_pred

# Check the accuracy of the model
print("Accuracy is ", accuracy_score(y_test,y_pred)*100)

As you can see, our decision tree model has an accuracy of around 73%. Now let's look at the structure of the tree to see which features it used to make decisions.


In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt

plt.figure(figsize=(15,10))
tree.plot_tree(clf_entropy, feature_names=names[0:8], class_names=['No Diabetes', 'Diabetes'], filled=True)
plt.show()

As you can see, the decision tree has used some features like 'plas' (Plasma glucose concentration) and 'mass' (Body mass index) to make decisions, but it has ignored other features like 'pres' (Diastolic blood pressure) and 'skin' (Triceps skinfold thickness). This is an example of how decision trees perform automatic feature selection.

## 2. Handling Missing Data
Now let's look at how to handle missing data. We'll create a new dataset with some missing values and then use different strategies to fill in those missing values.


In [ ]:
# Create a simple dataframe with some missing values
data = pd.DataFrame({
    'A': ['Yes', 'No', 'Yes', np.nan, 'No', 'Yes'],
    'B': ['No', 'Yes', 'Yes', 'No', 'Yes', np.nan],
    'C': [1, 2, np.nan, 4, 5, 6]
})
data

As you can see, the feature 'A' has a missing value in the fourth row, 'B' has a missing value in the last row, and 'C' has a missing value in the third row. Let's use different strategies to fill in these missing values.

### 2.1 Common Option for Categorical Data
We can use the `fillna()` function in pandas to fill in the missing values with the most common value in each column.


In [ ]:
data['A'].fillna(data['A'].mode()[0], inplace=True)
data['B'].fillna(data['B'].mode()[0], inplace=True)
data

As you can see, the missing value in 'A' has been replaced with 'Yes', which is the most common value in that column, and the missing value in 'B' has been replaced with 'No'.

### 2.3 Mean/Median for Continuous Data
For the continuous feature 'C', we could replace the missing value with the mean or median. Let's use the mean in this case.


In [ ]:
data['C'].fillna(data['C'].mean(), inplace=True)
data

The missing value in 'C' has been replaced with the mean of the other values in that column.

### 2.3 Regression for Continuous Data
If we had another continuous feature that was highly correlated with 'C', we could use regression to predict the missing value. However, since this is a simple example with only one continuous feature, we cannot demonstrate that here.

In conclusion, decision trees provide automatic feature selection and offer various ways to handle missing data. This makes them a powerful tool for machine learning tasks.

## References

1. L. Breiman, J. Friedman, R. Olshen, and C. Stone. Classification and Regression Trees. Wadsworth, Belmont, CA, 1984.
2. T. Hastie, R. Tibshirani and J. Friedman. Elements of Statistical Learning, Springer, 2009.
3. L. Breiman. Random Forests. Machine Learning, 45(1), 5-32, 2001.